### Utilizing the LLMs from AI Core for generating embeddings using REST APIs and interacting with the SAP HanaDB

In this guide, we illustrate a step-by-step procedure for utilizing the proxy LLMs deployed within the AI Core to generate embeddings for selected unstructured data. Here we will use the llm-commons REST API for genereating our embeddings. 

 This data is subsequently stored in the SAP Hana Database. Following this, we convert these created embeddings into a new data type: 'REAL_VECTOR', by leveraging the SAP HANA Vector Engine.

 Note: Alternatively, to use the llm-commons Python-sdk insted, refere to: 
 * Generate-and-store-embeddings_with-HanaDB-AICore-
 PythonSDK.ipynb

#### Pre-requisite: 

Use the secrets folder to store your service key credentials. Credentials required for:  
* Access to the SAP GenAI XL
* Access to the HanaDB


#### Step-by-step guide:
* Loading your data from csv 
* Connection with Hana database
* Create a new Hana table and push data into it
* Connection with AI Core proxy LLMs through REST API
* Generate embeddings for the desired column using text-embedding-ada-002 LLM 
* Add a new column of data type REAL_VECTOR to your data table ((Let us call this column: VECTOR_RE))
* Use the TO_REAL_VECTOR function to convert the embeggings to Real Vectors (VECTOR_RE). (This is necessary for the HANA DB to understand the embeddings.) and Update the data table.
* Decode the Real_Vector datatype back to embeddings: VECTOR_STR


#### 1. Loading your data

In [1]:
import pandas as pd

# get a csv file into pandas df
df = pd.read_csv('./data/INIS_NEWS_APPLICATION.csv', sep=';', quotechar='"', low_memory=False)
df.head(3)


,Key,Date,No,TopicNo,TopicID,TopicName,Domain,arXivID,Base,Link,SenderHTML,SenderName,Title,Abstract
0,2023.11.01.13.20.01,01.11.2023,1,1,Foundation,"Large Language and Foundation Models, Multi Mo...",arxiv,2310.18752,http://export.arxiv.org/rss/cs,http://arxiv.org/pdf/2310.18752.pdf,"<a href=""http://arxiv.org/find/cs/1/au:+Sui_G...","Guanghu Sui,Zhishuai Li,Ziyue Li,Sun Yang,Jing...",Reboost Large Language Model-based Text-to-SQL...,The previous state-of-the-art (SOTA) method ac...
1,2023.11.01.13.20.02,01.11.2023,2,1,Foundation,"Large Language and Foundation Models, Multi Mo...",arxiv,2310.20034,http://export.arxiv.org/rss/cs,http://arxiv.org/pdf/2310.20034.pdf,"<a href=""http://arxiv.org/find/cs/1/au:+Graul...","Moritz A. Graule,Volkan Isler",GG-LLM: Geometrically Grounding Large Language...,A robot in a human-centric environment needs t...
2,2023.11.01.13.20.03,01.11.2023,3,1,Foundation,"Large Language and Foundation Models, Multi Mo...",arxiv,2310.20329,http://export.arxiv.org/rss/cs,http://arxiv.org/pdf/2310.20329.pdf,"<a href=""http://arxiv.org/find/cs/1/au:+Hu_Q/...","Qisheng Hu,Kaixin Li,Xu Zhao,Yuxi Xie,Tiedong ...",InstructCoder: Empowering Language Models for ...,Code editing encompasses a variety of pragmati...


#### 2. Connection to HANA Database

In [2]:
import json
with open('data/secrets/ies-hana-vectordb-schema-poc-sk.json', 'r') as f:
    hana_service_key = json.load(f)

In [3]:
import hana_ml
from hana_ml import ConnectionContext

# cc = ConnectionContext(userkey='VDB_BETA', encrypt=True)
cc= ConnectionContext(
    address=hana_service_key['host'],
    port=hana_service_key['port'],
    user=hana_service_key['user'],
    password=hana_service_key['password'],
    currentSchema=hana_service_key['schema'],
    encrypt=True
    )
print(cc.hana_version())
print(cc.get_current_schema())

ModuleNotFoundError: No module named 'shapely'


4.00.000.00.1710842063 (CE2024.10)
USR_5SKS2ZNTSKBBRAFPULSZIT6NR


#### 3. Create a new Hana table and push data into it

In [4]:
# Create a Hana table
cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE "NEWS_APPL" (
    "Key" NVARCHAR(5000),
    "Date" NVARCHAR(5000),
    "No" BIGINT,
    "TopicNo" BIGINT,
    "TopicID" NVARCHAR(5000),
    "TopicName" NVARCHAR(5000),
    "Domain" NVARCHAR(5000),
    "arXivID" DOUBLE,
    "Base" NVARCHAR(5000),
    "Link" NVARCHAR(5000),
    "SenderHTML" NVARCHAR(5000),
    "SenderName" NVARCHAR(5000),
    "Title" NVARCHAR(5000),
    "Abstract" NCLOB MEMORY THRESHOLD 0)'''
cursor.execute(sql_command)
cursor.close()

In [5]:
# import dataframe into hana table
from hana_ml.dataframe import create_dataframe_from_pandas

v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df,
    table_name="NEWS_APPL",
    allow_bigint=True,
    append=True,
    force=False)

100%|██████████| 1/1 [00:00<00:00,  3.00it/s]


#### 4. Connection with AI Core proxy LLMs through REST API

In [6]:
# Read ML deployed model
with open('data/secrets/genai-xl-test-instance.json') as f:
    sk = json.load(f)

In [7]:

import requests
import os

response = requests.post(
    sk['url']+"/oauth/token",
    data={"grant_type": "client_credentials"},
    auth=(sk['clientid'], sk['clientsecret']),
    timeout=8000,
)
auth_token = response.json()["access_token"]

In [8]:
deployment_url="https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/da6a26f83dc8e241"

#### 5. Generate embeddings for the desired column using text-embedding-ada-002 LLM (REST_API)

In [9]:

def get_embedding(input, model="text-embedding-ada-002") -> str:
    # Preparing the input for inference

    test_input = {
        "model" : "text-embedding-ada-002",
        "input" : input
    }
    endpoint = f"{deployment_url}/embeddings?api-version=2023-05-15" # endpoint implemented in serving engine
    print(endpoint)
    headers = {"Authorization": f"Bearer {auth_token}",
               'ai-resource-group': "da6a26f83dc8e241",
               "Content-Type": "application/json"}
    response = requests.post(endpoint, headers=headers, json=test_input)
    response_json = response.json()
    return response_json['data'][0]['embedding']

In [10]:
# get some data from the newly created table: 
# you could join multiple text columns in SQL
# Here we are selecting the Top 10 "Key"

hdf = cc.sql('''SELECT TOP 10 "Key", "Abstract" FROM NEWS_APPL''')
df_abstract = hdf.collect()

In [11]:
# generate embeddings from the text
rows = []
for index, row in df_abstract.iterrows():
    text = row['Abstract']
    try:
        text_vector = get_embedding(input=text)
        print(text_vector)
        # text_vector = '[0, 1]'
        myrow = (str(text_vector), row['Key'])
        rows.append(myrow)
    except Exception as e:
        print(e)

https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/da6a26f83dc8e241/embeddings?api-version=2023-05-15
[-0.010810758, 0.013520504, 0.0062592314, -0.021099325, -0.028282976, 0.03344843, -0.015679834, -0.002965549, -0.041662347, -0.04919883, -0.002429245, 0.024641754, -0.0099780755, 0.010535549, 0.0074447454, 0.022101367, 0.0206477, -0.0027238594, 0.009413545, -0.01604678, -0.0023657354, -0.0070072343, -0.018375466, 0.022538878, -0.013915676, -0.01443081, 0.016413724, -0.034012962, -0.021014646, 0.014931831, 0.005161925, -0.0062133633, -0.02817007, -0.004646791, -0.025248624, 0.0069225547, -0.010098038, 0.00094823475, 0.022256613, -0.018050862, 0.013576957, 0.015355228, 0.005828777, -0.026462365, 0.0068449317, 0.0119680455, -0.0019088186, -0.021734422, 0.003725901, 0.0018700071, 0.014247337, 0.021677969, -0.03994053, 0.0057546822, -0.000962348, 0.0033077956, 0.02726682, 0.027817238, 0.016456064, 8.407314e-05, 0.008559694, 0.017161727, -0.018431919, 0.040307

#### 6. Add a new column of data type REAL_VECTOR to your data table ((Let us call this column: VECTOR_RE))

In [12]:
# # add a vector column to your table with datatype REAL_VECTOR
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE NEWS_APPL ADD (VECTOR_RE REAL_VECTOR(1536))'''
cursor.execute(sql_command)
cursor.close()

#### 7. Use the TO_REAL_VECTOR function to convert the embeggings to Real Vectors (VECTOR). (This is necessary for the HANA DB to understand the embeddings.) and Update the data table.

In [13]:
# bulk update
cc.connection.setautocommit(False)
cursor = cc.connection.cursor()
sql = 'UPDATE NEWS_APPL SET VECTOR_RE = TO_REAL_VECTOR(?) WHERE "Key" = ?'
try:
    print(sql)
    print(rows[0])
    cursor.executemany(sql, rows)
except Exception as e:
    cc.connection.rollback()
    print("An error occurred:", e)
try:
    cc.connection.commit()
finally:
    cursor.close()
cc.connection.setautocommit(True)

UPDATE NEWS_APPL SET VECTOR_RE = TO_REAL_VECTOR(?) WHERE "Key" = ?
('[-0.010810758, 0.013520504, 0.0062592314, -0.021099325, -0.028282976, 0.03344843, -0.015679834, -0.002965549, -0.041662347, -0.04919883, -0.002429245, 0.024641754, -0.0099780755, 0.010535549, 0.0074447454, 0.022101367, 0.0206477, -0.0027238594, 0.009413545, -0.01604678, -0.0023657354, -0.0070072343, -0.018375466, 0.022538878, -0.013915676, -0.01443081, 0.016413724, -0.034012962, -0.021014646, 0.014931831, 0.005161925, -0.0062133633, -0.02817007, -0.004646791, -0.025248624, 0.0069225547, -0.010098038, 0.00094823475, 0.022256613, -0.018050862, 0.013576957, 0.015355228, 0.005828777, -0.026462365, 0.0068449317, 0.0119680455, -0.0019088186, -0.021734422, 0.003725901, 0.0018700071, 0.014247337, 0.021677969, -0.03994053, 0.0057546822, -0.000962348, 0.0033077956, 0.02726682, 0.027817238, 0.016456064, 8.407314e-05, 0.008559694, 0.017161727, -0.018431919, 0.040307473, -0.017712144, -0.014917717, -0.009568791, 0.005168982, 0.009

#### 8. Decode the Real_Vector datatype bact to embeddings: VECTOR_STR

In [14]:
# The TO_NVARCHAR function is used here to decode REAL_VECTORS into vector embeddings

hdf = cc.sql('''SELECT TOP 10 "Key", "Abstract", TO_NVARCHAR(VECTOR_RE) AS VECTOR_STR FROM NEWS_APPL WHERE VECTOR_RE IS NOT NULL''')
df_abstract = hdf.collect()
df_abstract

,Key,Abstract,VECTOR_STR
0,2023.11.01.13.20.01,The previous state-of-the-art (SOTA) method ac...,"[-0.010810758,0.013520504,0.0062592314,-0.0210..."
1,2023.11.01.13.20.02,A robot in a human-centric environment needs t...,"[-0.012119322,-0.009318922,-0.0070246183,-0.01..."
2,2023.11.01.13.20.03,Code editing encompasses a variety of pragmati...,"[-0.0073093185,0.017813439,0.0047368812,-0.028..."
3,2023.11.01.13.20.04,Transformer neural networks show promising cap...,"[-0.012066817,0.017473692,-0.0036241987,-0.036..."
4,2023.11.01.13.20.05,The critical point I am making in this article...,"[-0.0068290434,-0.027423717,-0.00043647736,-0...."
5,2023.11.01.13.20.06,The future of intelligent manufacturing machin...,"[-0.024562826,0.0011631983,-0.020067578,-0.007..."
6,2023.11.01.13.20.07,This paper presents a deep learning based appr...,"[-0.003260679,0.016664552,0.016116826,-0.03195..."
7,2023.11.01.13.20.08,"In recent years, the rapid growth of online mu...","[-0.029618578,-0.0132206315,0.012463803,-0.063..."
8,2023.11.01.13.20.09,"Under the ""dual carbon"" target in China, virtu...","[-0.0074294377,-0.01571948,0.013346094,-0.0039..."
9,2023.11.01.13.20.10,Load shapes derived from smart meter data are ...,"[0.009132967,0.012615126,0.006297301,-0.036449..."
